In [1]:
import pandas as pd
import re
df = pd.read_csv('ModelResults.csv')  #用实际的文件名替换

In [2]:
# 删除列,
df = df.drop(columns=['tag','backdoor_num'])
df['backdoor_label'] = df['backdoor_label'].apply(lambda x: len(re.findall(r'\d+', x)) if isinstance(x, str) else 0)


In [3]:
df = df.fillna(0)

In [4]:
#越小越优指标位置
index=['backdoor_label','trigger_std','trigger_size','PoisonSR',] 
for row in index:
    df[row]=max(df[row])-df[row]

In [5]:
#归一化
ymin = 0.002
ymax = 1

def normalize(column):
    return (column - column.min()) / (column.max() - column.min()) * (ymax - ymin) + ymin

df_norm = df.apply(normalize)

In [6]:
df_norm

,ACC-0.005,pgd-0.005,After_Datapoison_Defense_ACC,PoisonSR,NoisyACC-0.005,trigger,trigger_std,backdoor_label,nifgsm-0.005,trigger_size,...,BlurredACC-0.01,sinifgsm-0.005,vmifgsm-0.005,difgsm-0.005,NoisyACC-0.01,BlurredACC-0.005,CompressedACC-0.005,fgsm-0.005,CompressedACC-0.01,tifgsm-0.005
0,0.935957,0.839032,0.703297,1.000,0.022367,1.000,1.000000,0.002,1.0000,1.000000,...,0.97505,1.000000,1.000000,1.000000,0.059029,0.820872,0.552621,0.906438,0.607929,1.000000
1,0.935957,0.839032,0.691248,0.002,0.765776,0.002,0.092410,1.000,1.0000,0.002000,...,1.00000,1.000000,1.000000,1.000000,0.757629,0.820872,0.776310,0.906438,0.875250,1.000000
2,0.850567,0.774645,0.996851,0.002,0.709765,0.002,0.002000,0.002,1.0000,0.120447,...,0.80040,1.000000,0.956609,0.956609,0.729114,0.513795,0.690276,0.812875,0.714857,0.844063
3,0.935957,0.839032,0.997946,0.002,0.821786,0.002,0.002000,0.002,1.0000,0.793929,...,1.00000,1.000000,1.000000,1.000000,0.776638,0.820872,0.776310,0.906438,0.875250,1.000000
4,0.722481,0.694161,0.031438,1.000,0.760684,0.002,0.130447,1.000,0.6507,0.911428,...,0.70060,0.589059,0.826435,0.826435,0.743371,0.360256,0.759103,0.750500,0.697036,0.625750
5,0.642428,0.613677,0.500658,1.000,1.000000,0.002,0.002000,1.000,1.0000,0.858169,...,0.22655,1.000000,0.913217,0.913217,1.000000,0.309077,1.000000,0.641344,1.000000,0.469813
6,0.914610,0.758548,0.419328,0.002,0.872704,0.002,0.124877,1.000,0.9002,0.812024,...,0.90020,0.882588,0.913217,0.913217,0.857429,0.590564,0.931172,0.828469,0.857429,0.594562
7,0.850567,0.774645,0.683033,0.002,0.709765,0.002,0.008406,1.000,1.0000,0.687869,...,0.80040,1.000000,0.956609,0.956609,0.672086,0.513795,0.690276,0.812875,0.714857,0.844063
8,0.701134,0.822935,0.720412,0.002,0.078378,0.002,0.041716,0.002,0.2515,0.128487,...,0.90020,0.178118,0.175565,0.175565,0.011505,1.000000,0.156862,0.797281,0.002000,0.313875
9,0.887925,0.919516,0.686319,0.002,0.139480,0.002,0.062517,1.000,0.5509,0.787908,...,0.55090,0.471647,0.435913,0.435913,0.092295,1.000000,0.139655,0.890844,0.091107,0.438625


In [7]:
import numpy as np

# 假设df是你的dataframe
m = len(df) 

# 计算信息熵
E = -1/np.log(m) * (df_norm*np.log(df_norm)).sum()

# Normalize entropy
E_norm = E.values / np.log(m)

# 计算权重
w = (1 - E_norm) / sum(1 - E_norm)

In [8]:
E_norm

array([0.27120043, 0.33011129, 0.40499017, 0.01427697, 0.33161224,
       0.02320008, 0.25527829, 0.00892311, 0.26714461, 0.37835557,
       0.29677642, 0.26893469, 0.25901065, 0.25593647, 0.32553181,
       0.24147927, 0.25349641, 0.25593647, 0.32219571, 0.44511416,
       0.43378829, 0.28938766, 0.35936664, 0.38956428])

In [9]:
w = pd.Series(w, index=df.columns)

# 利用 Pandas 的 apply 方法进行乘法操作
scores = df_norm * w

# 对每行数据进行求和以得到一个总分
total_scores = scores.sum(axis=1)

In [10]:
total_scores

0     0.809534
1     0.749455
2     0.647006
3     0.729417
4     0.642982
5     0.698240
6     0.714457
7     0.711161
8     0.320887
9     0.494408
10    0.264338
11    0.313633
12    0.156731
13    0.572669
dtype: float64

In [11]:
w.to_csv("./weight.csv")